# training

## setup

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/tiger_detection/

/content/drive/MyDrive/tiger_detection


In [4]:
!pip3 install keras kimm -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [5]:
import numpy as np
import keras
from keras import layers
import kimm
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

## prepare train and test

In [6]:
# Define paths to train and test datasets
train_dir = "images/train"
test_dir = "images/test"
test2_dir = "images/test2"

# Create training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='int',
    shuffle=True
)

# Create testing dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='int',
    shuffle=False
)
test2_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test2_dir,
    label_mode='int',
    shuffle=False
)

# we need to unbatch because there's somehow an unwanted additional dimension
train_ds = train_ds.unbatch()
test_ds = test_ds.unbatch()
test2_ds = test2_ds.unbatch()

print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of test samples: {test_ds.cardinality()}")

Found 6000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 320 files belonging to 1 classes.
Number of training samples: -2
Number of test samples: -2


In [7]:
print(train_ds.element_spec)

(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))


In [8]:
print(test_ds.element_spec)

(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))


In [9]:
# Setup dataset with tf.data
resize_fn = keras.layers.Resizing(224, 224)

train_ds = train_ds.map(lambda x, y: (resize_fn(x), y))
test_ds = test_ds.map(lambda x, y: (resize_fn(x), y))
test2_ds = test2_ds.map(lambda x, y: (resize_fn(x), y))

batch_size = 16
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
test2_ds = test2_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

In [10]:
# kimm will load weights pre-trained on ImageNet by default in most cases
# You can try different model such as ConvNeXtAtto, EfficientNetLiteB0, GhostNet100, RegNetY002, etc.
base_model = kimm.models.EfficientNetV2B0(
    input_shape=(224, 224, 3),
    include_preprocessing=True,
    include_top=False,
)

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = inputs

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

29451563/29451563 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_1 (InputLayer)          │ (None, 224, 224, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ EfficientNetV2B0 (EfficientNetV2B0) │ (None, 7, 7, 1280)           │     5,919,312 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d            │ (None, 1280)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dropout (Dropout)                   │ (None, 1280)                 │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense (Dense)                       │ (None, 1)                    │         1,281 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 5,920,593 (22.59 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

## training

In [11]:
# You should achieve good results in 2 epochs
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
print("Fitting the top layer of the model")
model.fit(train_ds, epochs=epochs)#, validation_data=validation_ds)

result = model.evaluate(test_ds)
print(f"Test accuracy: {result[1] * 100:3.2f}%")

Fitting the top layer of the model
Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 597s 2s/step - binary_accuracy: 0.6108 - loss: 0.6107
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 17ms/step - binary_accuracy: 0.9313 - loss: 0.2647
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - binary_accuracy: 0.9465 - loss: 0.1795
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - binary_accuracy: 0.9463 - loss: 0.1457
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - binary_accuracy: 0.9532 - loss: 0.1257
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - binary_accuracy: 0.9555 - loss: 0.1127
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - binary_accuracy: 0.9575 - loss: 0.1045
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - binary_accuracy: 0.9601 - loss: 0.0972
Epoch 9/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - binary_accuracy: 0.9582 - loss: 0.0926
Epoch 10/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - binary_accuracy: 0.9615 - loss: 0.0864
125/125 ━━━━━━━━━━━━━━━━━━━━ 396s 3s/step - binary_accuracy: 0.9790 - loss: 0.0678
Test accuracy: 97.10%


In [12]:
result2 = model.evaluate(test2_ds)
print(f"Test2 accuracy: {result2[1] * 100:3.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - binary_accuracy: 0.8809 - loss: 0.3343
Test2 accuracy: 87.50%
